In [1]:
# ![alt text](problem_definition.png "Problem definition")
# ![alt text](problem_variables.png "Problem variables")
# <h2> References </h2>
# <h4>1) G. Yao, J. Bi, Y. Li, and L. Guo, “On the Capacitated Controller Placement Problem in Software Defined Networks,” IEEE Commun. Lett., 18, 4 (2014) </h4>
# <h4>2) F. A. Ozsoy and M. C. Pinar, “An exact algorithm for the capacitated vertex
# k-center problem,” Comput. Oper. Res., vol. 33, no. 5, pp. 1420–1436,
# May 2006.</h4>
# ![alt text](algorithm_pseudocode.png)

In [3]:
import import_ipynb
import networkx as nx
from graph_utils import Graph
from custom_ccpp_solver import CustomCCPPSolver
from cplex_ccpp_solver import CPLEXCCPPSolver
import numpy as np
import matplotlib.pyplot as plt
import time

rnd = np.random
rnd.seed()
    
def getAllVariables(graph, K):
    G = graph.getGraph()
    V_count = len(G.nodes)   
    K_initial = V_count
    V = [list(G.nodes).index(i) for i in G.nodes]      

    theta = [i for i in range(0, K_initial)]            
    l_i = {i : 10 for i in V}   

    # generate different capacity values for controllers that sum up to the sum of capacity of all switches multiplied by a constant
    # L_values_for_theta = rnd.dirichlet(np.ones(K), size=1).flatten()*sum(l_i.values())*2
    L  = {i : 100 for i in V for i in theta}
    
    V_loc_x, V_loc_y = graph.getGraphNodeCoordinates(True)

    E = [(list(G.nodes).index(i),list(G.nodes).index(j)) for i, j in G.edges]  

    # calculate the weight of each edge in graph as euclidean distance between two nodes
    for i, j in G.edges:
        ii = list(G.nodes).index(i)
        jj = list(G.nodes).index(j)
        G[i][j]['weight'] = np.hypot(V_loc_x[ii] - V_loc_x[jj], V_loc_y[ii] - V_loc_y[jj])

    # calculate matrix representing shortest distance between all nodes 
    d =  nx.floyd_warshall_numpy(G, weight='weight')

    # possibilities of placing controller theta in place of a switch v
    TV = [(i, j) for i in theta for j in V]

    # each node to each node dictionary
    SV = [(s, v) for s in V for v in V]
    return (K, V, V_count, theta, L, l_i, V_loc_x, V_loc_y, E, d, TV, SV, G)

def getKValuesToTest(nr_of_nodes):
    return [np.ceil(nr_of_nodes/3), np.ceil(nr_of_nodes/4), np.ceil(nr_of_nodes/6), np.ceil(nr_of_nodes/8)]

graph_files = ['Netrail.gml','Poland.gml', 'Germany.gml', 'Usa.gml']
#graph_files = ['Usa.gml']
for graph_file in graph_files:
    graph = Graph()
    graph.readGraph(graph_file)
    #graph.showGraph()
    nr_of_nodes = len(graph.getGraph().nodes())
    k_values = getKValuesToTest(nr_of_nodes)
    print("Graph: " + graph_file + ", nr of nodes: " + str(nr_of_nodes))
    for k in k_values:        
        params = getAllVariables(graph, k)
        customAlgorithm = CustomCCPPSolver(params)
        start = time. time()
        customAlgorithm.solve()
        end = time. time()
        print("------ Custom algorithm: k " + str(k) + ", time to solve:", str(end - start))
        
        cplexAlgorithm = CPLEXCCPPSolver(params)
        start = time. time()
        cplexAlgorithm.solve()
        end = time. time()
        print("------ CPLEX  algorithm: k " + str(k) + ", time to solve:", str(end - start))
        print("")
        #customAlgorithm.plotSolution()

Graph: Netrail.gml, nr of nodes: 7
------ Custom algorithm: k 3.0, time to solve: 0.11674165725708008
------ CPLEX  algorithm: k 3.0, time to solve: 0.028701066970825195

------ Custom algorithm: k 2.0, time to solve: 0.10206079483032227
------ CPLEX  algorithm: k 2.0, time to solve: 0.02911376953125

------ Custom algorithm: k 2.0, time to solve: 0.10252046585083008
------ CPLEX  algorithm: k 2.0, time to solve: 0.030922651290893555

------ Custom algorithm: k 1.0, time to solve: 0.10251975059509277
------ CPLEX  algorithm: k 1.0, time to solve: 0.02950739860534668

Graph: Poland.gml, nr of nodes: 26
------ Custom algorithm: k 9.0, time to solve: 0.4072580337524414
------ CPLEX  algorithm: k 9.0, time to solve: 0.3136179447174072

------ Custom algorithm: k 7.0, time to solve: 0.26323938369750977
------ CPLEX  algorithm: k 7.0, time to solve: 0.2883117198944092

------ Custom algorithm: k 5.0, time to solve: 0.3711674213409424
------ CPLEX  algorithm: k 5.0, time to solve: 0.308149099